In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
import math
%matplotlib inline
import matplotlib as plt

In [3]:
# 4 times 3 ggg and there can be 1-7 nucleotides in between. that nucleotide can be a G too.
# the minus ones are C. -/+ is the strand. + is the strand
# in the bed file is only the motif itself. we can look with deep learning more upstream and downstream

In [4]:
bases = dict(zip("ACGT", [0,1,2,3]))
bases 

{'A': 0, 'C': 1, 'G': 2, 'T': 3}

In [5]:
positive = pd.read_csv("data/g_quad_AAAA_unique_noSSE.bed", delimiter="\t", names=["chr", "start", "end",
                                                                                     "seq", "col", "strand"])
positive.head()

,chr,start,end,seq,col,strand
0,chr1,790094,790096,GGGATGGGATGGGATGGGATGGGATGGG,1111,+
1,chr1,790097,790113,GGGATGGGATGGGATGGGATGGGATGGG,1111,+
2,chr1,834091,834096,GGGGTTTGGGGGCTGGGGCCTGGGAGGG,1111,+
3,chr1,835046,835069,CCCTCTCCCTTGCCTCCCTCCCC,1111,-
4,chr1,844622,844631,GGGTCTGCGGGGAGTAGGGTGGGG,1111,+


In [6]:
negative = pd.read_csv("data/g_quad_NovaSeq_SSEs_50bp_clean.bed", delimiter="\t", names=["chr", "start", "end",
                                                                                     "seq", "col", "strand"])
negative.head()


,chr,start,end,seq,col,strand
0,chr1,904296,904299,GGGTGGGGGGGTGGGGGGCGGCATCGGG,1111,+
1,chr1,937401,937411,CCCCCCCCCCCACCC,1111,-
2,chr1,955891,955916,CCCTACCCCCCTTCACCCCCTCCCC,1111,-
3,chr1,984237,984268,GGGCCTGGGGGGCAAGTCGGGGGGCGGGGGG,1111,+
4,chr1,984273,984299,GGGCAGGGTCCCCTGGGAGGATGGGG,1111,+


In [7]:
def verify_data(df):
    wrong_data = []
    good_data = []
    
    def filter_wrong_and_good_data(row):
        diff = row["end"] - row["start"]
        if diff is not len(row["seq"]):
            wrong_data.append(row.name)
        else:
            good_data.append(row.name)
    
    df.apply(filter_wrong_and_good_data, axis = 1)
    
    return wrong_data, good_data


In [8]:
(len(verify_data(negative)[0]), len(verify_data(negative)[1]))


(4085, 11804)

In [9]:
negative_good_data = negative.drop(verify_data(negative)[0])
negative_good_data.head()


,chr,start,end,seq,col,strand
2,chr1,955891,955916,CCCTACCCCCCTTCACCCCCTCCCC,1111,-
3,chr1,984237,984268,GGGCCTGGGGGGCAAGTCGGGGGGCGGGGGG,1111,+
4,chr1,984273,984299,GGGCAGGGTCCCCTGGGAGGATGGGG,1111,+
6,chr1,1035169,1035192,GGGGGGGGGGGGGTGGGCAGGGG,1111,+
7,chr1,1035208,1035229,GGGCAAAGGGATGGGACAGGG,1111,+


In [12]:
(len(verify_data(positive)[0]), len(verify_data(positive)[1]))



(12285, 154544)

In [14]:

positive_good_data = positive.drop(verify_data(positive)[0])
positive_good_data.head()


,chr,start,end,seq,col,strand
3,chr1,835046,835069,CCCTCTCCCTTGCCTCCCTCCCC,1111,-
11,chr1,876250,876281,CCCACACTCGCCCACACTCCCCCACACTCCC,1111,-
17,chr1,902899,902925,GGGGACGAGGGGGCCCGGGATGCGGG,1111,+
18,chr1,904022,904047,CCCCTGCAGACCCTGTGCCCAGCCC,1111,-
19,chr1,904114,904140,GGGAGGCCTGGGGCGGAGGGCCGGGG,1111,+


In [15]:
def get_window_start(row):
    seq = row["seq"]
    length = len(seq)
    result = row["start"] - 200
    middle = length // 2
    result += middle
    return result


def get_window_end(row):
    seq = row["seq"]
    length = len(seq)
    result = row["end"] + 200
    middle = length / 2
    if length % 2 == 0:
        result -= middle
    else:
        result -= math.ceil(middle)  
    return int(result)
    
    
def get_ranges(df):
        
    df["start_new"] = df.apply(get_window_start, axis=1)
    df["end_new"] = df.apply(get_window_end, axis=1)
    
    return df


In [17]:
positive_good_data_with_ranges = get_ranges(positive_good_data)
positive_good_data_with_ranges.head()

,chr,start,end,seq,col,strand,start_new,end_new
3,chr1,835046,835069,CCCTCTCCCTTGCCTCCCTCCCC,1111,-,834857,835257
11,chr1,876250,876281,CCCACACTCGCCCACACTCCCCCACACTCCC,1111,-,876065,876465
17,chr1,902899,902925,GGGGACGAGGGGGCCCGGGATGCGGG,1111,+,902712,903112
18,chr1,904022,904047,CCCCTGCAGACCCTGTGCCCAGCCC,1111,-,903834,904234
19,chr1,904114,904140,GGGAGGCCTGGGGCGGAGGGCCGGGG,1111,+,903927,904327


In [18]:
def check_data_consistent(df):
    all_good = True
    
    def check(row):
        nonlocal all_good 
        if (row["end_new"] - row["start_new"]) != 400:
            all_good = False
            print("{} is not ok".format(row.name))
    
    df.apply(check, axis=1)
    
    if all_good:
        print("all good")
    else:
        print("wrong data")


In [19]:
check_data_consistent(positive_good_data_with_ranges)


all good


In [20]:
negative_good_data_with_ranges = get_ranges(negative_good_data)
negative_good_data_with_ranges.head()


,chr,start,end,seq,col,strand,start_new,end_new
2,chr1,955891,955916,CCCTACCCCCCTTCACCCCCTCCCC,1111,-,955703,956103
3,chr1,984237,984268,GGGCCTGGGGGGCAAGTCGGGGGGCGGGGGG,1111,+,984052,984452
4,chr1,984273,984299,GGGCAGGGTCCCCTGGGAGGATGGGG,1111,+,984086,984486
6,chr1,1035169,1035192,GGGGGGGGGGGGGTGGGCAGGGG,1111,+,1034980,1035380
7,chr1,1035208,1035229,GGGCAAAGGGATGGGACAGGG,1111,+,1035018,1035418


In [21]:
check_data_consistent(negative_good_data_with_ranges)



all good


In [23]:
from Bio.Seq import Seq
from Bio import SeqIO


records = list(SeqIO.parse("./data/genome.fa", "fasta"))
len(records)



2580

In [74]:
def add_new_seq(df):
    def new_seq(row):
        return str(records[0].seq[row["start_new"]: row["end_new"]])
        
    df["seq_new"] = df.apply(new_seq, axis=1)
    
    return df


In [75]:
negative_with_seq = add_new_seq(negative_good_data_with_ranges)
negative_with_seq.head()

,chr,start,end,seq,col,strand,start_new,end_new,seq_new
2,chr1,955891,955916,CCCTACCCCCCTTCACCCCCTCCCC,1111,-,955703,956103,TCCCTTTTCAGGAAGGAAAGAAGGTGGGGCCGCTCCAACTGGCCCC...
3,chr1,984237,984268,GGGCCTGGGGGGCAAGTCGGGGGGCGGGGGG,1111,+,984052,984452,TGGAGTTGGCCTGAGGCTTCAGGGGAAGCCCTTCCCTGTATCCAGC...
4,chr1,984273,984299,GGGCAGGGTCCCCTGGGAGGATGGGG,1111,+,984086,984486,CCTGTATCCAGCCCAGTCATGACCCTTCCTGGTGGGAGGGTGGCTG...
6,chr1,1035169,1035192,GGGGGGGGGGGGGTGGGCAGGGG,1111,+,1034980,1035380,GCGGTGGACTCTTCCAGGGAAGGGGGTCCTGCCTGCACCCCTGTGG...
7,chr1,1035208,1035229,GGGCAAAGGGATGGGACAGGG,1111,+,1035018,1035418,CCCTGTGGCTGGGGCCCCATCTGACAGGGGTCAGGCCATGACTATT...


In [77]:
positive_with_seq = add_new_seq(positive_good_data_with_ranges)
positive_with_seq.head()

,chr,start,end,seq,col,strand,start_,end_,start_new,end_new,seq_new
3,chr1,835046,835069,CCCTCTCCCTTGCCTCCCTCCCC,1111,-,834857,835257,834857,835257,ATAACTAGATATACGAGTTTAAATAATTTTGTCAGAAACTGTTTCT...
11,chr1,876250,876281,CCCACACTCGCCCACACTCCCCCACACTCCC,1111,-,876065,876465,876065,876465,CCCCACACTCCCCCATACTCGCCCACACTCCCCCACACCCCACACT...
17,chr1,902899,902925,GGGGACGAGGGGGCCCGGGATGCGGG,1111,+,902712,903112,902712,903112,GCAGTGAGCCGAGATCGCGCCATTGCACCCAGCCTGGGCAACCAGA...
18,chr1,904022,904047,CCCCTGCAGACCCTGTGCCCAGCCC,1111,-,903834,904234,903834,904234,GGCCCTGAGGCTGGGCAAGGCTGTCCACCCCGCTGTCAGAACCCCA...
19,chr1,904114,904140,GGGAGGCCTGGGGCGGAGGGCCGGGG,1111,+,903927,904327,903927,904327,CCAGCGGGCACAAGGTTGGGGCAGCTCTGTTCCCAGCAGGCCGAGC...


In [78]:
negative_with_seq.to_csv("./data/negative_examples.csv")
positive_with_seq.to_csv("./data/positive_examples.csv")

In [37]:
negative_with_seq = pd.read_csv("./data/negative_examples.csv")
positive_with_seq = pd.read_csv("./data/positive_examples.csv")

In [41]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
integer_encoded_seq_pos = label_encoder.fit_transform(positive_with_seq["seq_new"])
integer_encoded_seq_pos

array([29926, 52855, 80304, ...,  4981, 85897, 19489])

In [43]:
onehot_encoder_pos = OneHotEncoder(sparse=False)
integer_encoded_seq_pos = integer_encoded_seq_pos.reshape(len(integer_encoded_seq_pos), 1)
onehot_encoded_seq_pos = onehot_encoder_pos.fit_transform(integer_encoded_seq_pos)
onehot_encoded_seq_pos

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [44]:
# LabelEncoder takes categorical data, like country names and encodes them into integers: 0,1,2
# but this can confuse the model by thinking that there is a mathematical relation 0 < 1
# so we now use one hot encoder to transform the data into a bunch of 0 and 1. it will split the array into multiple arrays. 

In [45]:
integer_encoded_seq_neg = label_encoder.fit_transform(negative_with_seq["seq_new"])
onehot_encoder_neg = OneHotEncoder(sparse=False)
integer_encoded_seq_neg = integer_encoded_seq_neg.reshape(len(integer_encoded_seq_neg), 1)
onehot_encoded_seq_neg = onehot_encoder_neg.fit_transform(integer_encoded_seq_neg)
onehot_encoded_seq_neg

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [48]:
Y_pos = [1] * len(onehot_encoded_seq_pos)
Y_neg = [0] * len(onehot_encoded_seq_neg)

In [49]:
features = np.concatenate((onehot_encoded_seq_pos, onehot_encoded_seq_neg), axis=0)
features

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [51]:
onehot_encoded_seq_pos.shape

(154544, 140723)

In [52]:
onehot_encoded_seq_neg.shape

(11804, 10825)

In [56]:
integer_encoded_seq_pos.shape

(154544, 1)

In [71]:
n = positive_with_seq["seq_new"].apply(lambda x: True if "N" in x else False)
np.where(n == True)

(array([ 15218,  15223,  15224, ..., 153676, 153677, 153678]),)

In [72]:
bases = dict(zip("ACGTN", [[1,0,0,0,0],[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1]]))
bases 


In [76]:
X_positive = []
def encode(row):
    result = []
    for base in row:
        result.append(bases[base])
        
    X_positive.append(result)
positive_with_seq["seq_new"].apply(encode)
X_positive[0][0]

In [81]:
X_negative = []
def encode_neg(row):
    result = []
    for base in row:
        result.append(bases[base])
        
    X_negative.append(result)
negative_with_seq["seq_new"].apply(encode_neg)
X_negative[0][0]

[0, 0, 0, 1, 0]

In [83]:
features = np.concatenate((X_positive, X_negative), axis=0)
features.shape

(166348, 400, 5)

In [85]:
d = [bases["A"],bases["C"]]
e = [bases["A"],bases["C"], bases["A"], bases["C"]]
e

[[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0]]

In [86]:
test_con = np.concatenate((d, e), axis=0)
test_con

array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0]])

In [87]:
test_con.shape

(6, 5)

In [122]:
Y_pos = [[1]] * len(X_positive)
Y_neg = [[0]] * len(X_negative)

In [123]:
labels = np.concatenate((Y_pos, Y_neg), axis=0)

In [124]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(features,
                                                    labels,
                                                    test_size=0.33,
                                                    random_state=42)


(111453, 400, 5)

In [125]:

X_train.shape

(111453, 400, 5)

In [2]:
plt.pyplot.figure(figsize=(5, 2))
plt.pyplot.matshow(X_train[999,:,:,0].transpose(), vmin=0, vmax=50, cmap=plt.pyplot.cm.coolwarm, fignum=0)


NameError: name 'X_train' is not defined

<Figure size 360x144 with 0 Axes>

TensorShape([Dimension(None), Dimension(400), Dimension(5)])

TensorShape([Dimension(50), Dimension(100)])

[[1], [1], [1]]

FileNotFoundError: File b'sales_data_training.csv' does not exist